In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
sns.set(color_codes=True)
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score



from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('genre_data.csv')
df

,Title,Author,Description,Genres1,Genres2,Genres3,Genres4,Genres5,Genres6,Genres7,Avg_Rating,Num_Ratings,URL
0,Married to the Bad Boy,Letty Scott,✔ Mind Blown ✔Action Packed ✔Heartthrob ✔Pag...,Romance,High School,Abuse,Young Adult,New Adult,Contemporary,Contemporary Romance,3.64,"1,238",https://www.goodreads.com/book/show/23601385-m...
1,Twisted,Anna Gallegos,✔Creepy ✔Heart Attack Inducing ✔Goosebumps w...,Horror,NaN,NaN,NaN,NaN,NaN,NaN,4.50,52,https://www.goodreads.com/book/show/25930928-t...
2,"Handcuffs, Kisses and Awkward Situations: Myst...",Olivia Harvard,✔Original ✔Mystery ✔Emotionally Satisfying ✔...,Romance,Young Adult,Fiction,Mystery,Teen,Young Adult Romance,NaN,4.12,"1,057",https://www.goodreads.com/book/show/23981700-h...
3,One Night with the Prince,T.M. Mendes,✔Royalty ✔Funny ✔Sexy ✔Romantic ✔Strong Hero...,Romance,Contemporary,Young Adult,Love,High School,Humor,Contemporary Romance,3.97,"1,430",https://www.goodreads.com/book/show/23255198-o...
4,My Beloved,T.M. Mendes,✔Spell binding ✔Sexy ✔Romantic ✔Monster Roma...,Vampires,Paranormal,Romance,Supernatural,Fantasy,Paranormal Romance,NaN,3.85,374,https://www.goodreads.com/book/show/24130436-m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7991,"Stand (Bleeding Stars, #6)",A.L. Jackson,Zachary Kennedy has never been known as a figh...,Romance,Contemporary Romance,Music,New Adult,Contemporary,Musicians,Audiobook,4.38,"4,346",https://www.goodreads.com/book/show/35181374-s...
7992,Girl at War,Sara Nović,"Zagreb, summer of 1991. Ten-year-old Ana Jurić...",Historical Fiction,Fiction,War,Historical,Contemporary,Young Adult,Coming Of Age,4.00,"29,392",https://www.goodreads.com/book/show/23209971-g...
7993,Between Two Worlds: Escape from Tyranny: Growi...,Zainab Salbi,Zainab Salbi was eleven years old when her fat...,Nonfiction,Memoir,Biography,History,Politics,Biography Memoir,Feminism,4.23,"2,932",https://www.goodreads.com/book/show/426973.Bet...
7994,"Blood for Blood (Ziba MacKenzie, #1)",Victoria Selman,Ziba Mackenzie profiles killers. Now one is pr...,Mystery,Thriller,Fiction,Crime,Mystery Thriller,Suspense,Amazon,3.78,"12,215",https://www.goodreads.com/book/show/40020563-b...


In [3]:
df.Genres1.unique()

array(['Romance', 'Horror', 'Vampires', 'Paranormal',
       'Historical Fiction', 'Fantasy', 'Urban Fantasy', 'Christian',
       'Leadership', 'Religion', 'Fiction', 'New Adult', 'Poetry',
       'Self Help', 'Nonfiction', 'Novels', 'Sociology', 'Young Adult',
       'Finance', 'Thriller', 'Psychology', 'Money',
       'Artificial Intelligence', 'Mystery', 'Christian Non Fiction',
       'Plays', 'Business', 'Short Stories', 'Picture Books', 'Comics',
       'History', 'Childrens', 'Classics', 'Spirituality', 'Philosophy',
       'Historical Romance', 'Middle Grade', 'Graphic Novels',
       'Biography', 'Contemporary', 'Essays', 'Science Fiction',
       'Theology', 'Christmas', 'M M Romance', 'Tarot', 'Holocaust',
       'Politics', 'Book Club', 'Realistic Fiction', 'Parenting', 'Humor',
       'Feminism', 'Christian Fiction', 'Memoir', 'True Crime',
       'Economics', 'Travel', 'Science', 'Music', 'BDSM', 'Dark',
       'Chick Lit', 'Manga', 'Romantic Suspense', 'Productivity',
 

In [4]:
df.Genres1.value_counts()

Fantasy               917
Nonfiction            895
Fiction               887
Classics              509
Historical Fiction    453
                     ... 
Magic                   1
Communication           1
Juvenile                1
Kids                    1
Epic Fantasy            1
Name: Genres1, Length: 205, dtype: int64

In [5]:
#df = df.loc[df['Genres1'].isin(['Nonfiction', 'Fiction', 'Fantasy'])]
df = df.loc[df['Genres1'].isin(['Fantasy', 'Historical', 'Classics', 'Young Adult', 'Mystery', 'Romance', 'Science Fiction', 'Thriller', 'Self Help', 'Horror'])]
df = df.groupby('Genres1').head(100)

In [6]:
df = df.reset_index()
df

,index,Title,Author,Description,Genres1,Genres2,Genres3,Genres4,Genres5,Genres6,Genres7,Avg_Rating,Num_Ratings,URL
0,0,Married to the Bad Boy,Letty Scott,✔ Mind Blown ✔Action Packed ✔Heartthrob ✔Pag...,Romance,High School,Abuse,Young Adult,New Adult,Contemporary,Contemporary Romance,3.64,"1,238",https://www.goodreads.com/book/show/23601385-m...
1,1,Twisted,Anna Gallegos,✔Creepy ✔Heart Attack Inducing ✔Goosebumps w...,Horror,NaN,NaN,NaN,NaN,NaN,NaN,4.50,52,https://www.goodreads.com/book/show/25930928-t...
2,2,"Handcuffs, Kisses and Awkward Situations: Myst...",Olivia Harvard,✔Original ✔Mystery ✔Emotionally Satisfying ✔...,Romance,Young Adult,Fiction,Mystery,Teen,Young Adult Romance,NaN,4.12,"1,057",https://www.goodreads.com/book/show/23981700-h...
3,3,One Night with the Prince,T.M. Mendes,✔Royalty ✔Funny ✔Sexy ✔Romantic ✔Strong Hero...,Romance,Contemporary,Young Adult,Love,High School,Humor,Contemporary Romance,3.97,"1,430",https://www.goodreads.com/book/show/23255198-o...
4,6,Demented (Sinister Tales Book 1),Anna Gallegos,"""Sometimes it's best to stay out where you're...",Horror,NaN,NaN,NaN,NaN,NaN,NaN,4.55,74,https://www.goodreads.com/book/show/25769260-d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,5729,The fifth agreement,Miguel Ruiz,"Since 1997, The Four Agreements has transforme...",Self Help,Spirituality,Nonfiction,Philosophy,Personal Development,Psychology,Inspirational,4.10,"13,425",https://www.goodreads.com/book/show/6979404-th...
902,5748,Snow in July,Kim Iverson Headlee,Sir Robert Alain de Bellencombre has been gran...,Historical,Paranormal,Fiction,Historical Fiction,Fantasy,Romance,Historical Romance,3.88,105,https://www.goodreads.com/book/show/22571859-s...
903,5874,Cheaper Than Therapy: How To Forgive and Overc...,Mykey Robinson,"Sticks and stones may break your bones, but WO...",Self Help,Nonfiction,NaN,NaN,NaN,NaN,NaN,4.35,20,https://www.goodreads.com/book/show/17926832-c...
904,5976,Codependent No More: How to Stop Controlling O...,Melody Beattie,The #1 bestseller that has helped heal million...,Self Help,Nonfiction,Psychology,Relationships,Mental Health,Personal Development,Health,4.12,"38,105",https://www.goodreads.com/book/show/720298.Cod...


In [7]:
df.dtypes

index            int64
Title           object
Author          object
Description     object
Genres1         object
Genres2         object
Genres3         object
Genres4         object
Genres5         object
Genres6         object
Genres7         object
Avg_Rating     float64
Num_Ratings     object
URL             object
dtype: object

In [8]:
df = df.astype({'Description':'string','Genres1':'string'})

In [9]:
df.dtypes

index            int64
Title           object
Author          object
Description     string
Genres1         string
Genres2         object
Genres3         object
Genres4         object
Genres5         object
Genres6         object
Genres7         object
Avg_Rating     float64
Num_Ratings     object
URL             object
dtype: object

In [10]:
df['Description']

0        ✔ Mind Blown ✔Action Packed ✔Heartthrob ✔Pag...
1        ✔Creepy ✔Heart Attack Inducing ✔Goosebumps w...
2        ✔Original ✔Mystery ✔Emotionally Satisfying ✔...
3        ✔Royalty ✔Funny ✔Sexy ✔Romantic ✔Strong Hero...
4       "Sometimes it's best to stay out where you're...
                             ...                        
901    Since 1997, The Four Agreements has transforme...
902    Sir Robert Alain de Bellencombre has been gran...
903    Sticks and stones may break your bones, but WO...
904    The #1 bestseller that has helped heal million...
905    Throughout our lives, many things may lead us ...
Name: Description, Length: 906, dtype: string

In [11]:
df['Genres1']

0         Romance
1          Horror
2         Romance
3         Romance
4          Horror
          ...    
901     Self Help
902    Historical
903     Self Help
904     Self Help
905    Historical
Name: Genres1, Length: 906, dtype: string

In [12]:
print('% of Missing Values for each Columns')
print('======================================')
df.apply(lambda x :(x.isnull().mean())*100)

% of Missing Values for each Columns


index           0.000000
Title           0.000000
Author          0.000000
Description     0.000000
Genres1         0.000000
Genres2         3.532009
Genres3         5.960265
Genres4         7.615894
Genres5         9.161148
Genres6        11.589404
Genres7        13.576159
Avg_Rating      0.000000
Num_Ratings     0.000000
URL             0.000000
dtype: float64

In [13]:
# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')

In [14]:
# Step - a : Remove blank rows if any.
df['Description'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
df['Description'] = [entry.lower() for entry in df['Description']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
df['Description']= [word_tokenize(entry) for entry in df['Description']]


In [15]:
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df['Description']):
    if index % 500 == 0:
      print(index)
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    df.loc[index,'desc_final'] = str(Final_words)

0
500


In [16]:
df['Description']

0      [✔, mind, blown, ✔action, packed, ✔heartthrob,...
1      [✔creepy, ✔heart, attack, inducing, ✔goosebump...
2      [✔original, ✔mystery, ✔emotionally, satisfying...
3      [✔royalty, ✔funny, ✔sexy, ✔romantic, ✔strong, ...
4      [``, sometimes, it, 's, best, to, stay, out, w...
                             ...                        
901    [since, 1997, ,, the, four, agreements, has, t...
902    [sir, robert, alain, de, bellencombre, has, be...
903    [sticks, and, stones, may, break, your, bones,...
904    [the, #, 1, bestseller, that, has, helped, hea...
905    [throughout, our, lives, ,, many, things, may,...
Name: Description, Length: 906, dtype: object

In [17]:
df['desc_final']

0      ['mind', 'blown', 'pack', 'turner', 'boy', 'ma...
1      ['attack', 'induce', 'worthy', 'two', 'friend'...
2      ['satisfy', 'turner', 'nora', 'montgomery', 'p...
3      ['heroine', 'hilarious', 'mix', 'princess', 'd...
4      ['sometimes', 'best', 'stay', 'welcome', 'deme...
                             ...                        
901    ['since', 'four', 'agreement', 'transform', 'l...
902    ['sir', 'robert', 'alain', 'de', 'bellencombre...
903    ['stick', 'stone', 'may', 'break', 'bone', 'wo...
904    ['bestseller', 'help', 'heal', 'million', 'rea...
905    ['throughout', 'life', 'many', 'thing', 'may',...
Name: desc_final, Length: 906, dtype: object

In [18]:
df['Description'][15]

['monsters',
 'are',
 'made',
 ',',
 'not',
 'born',
 '...',
 'most',
 'called',
 'him',
 'the',
 'jailor',
 '.',
 'some',
 'called',
 'him',
 'a',
 'monster',
 '.',
 'if',
 'he',
 'had',
 'a',
 'name',
 ',',
 'nobody',
 'had',
 'survived',
 'meeting',
 'him',
 'long',
 'enough',
 'to',
 'tell',
 'it',
 '.',
 'they',
 'said',
 'he',
 'sat',
 'atop',
 'a',
 'mound',
 'of',
 'skulls',
 'and',
 'marked',
 'each',
 'death',
 'on',
 'his',
 'throne',
 ',',
 'like',
 'notches',
 'on',
 'a',
 'bedpost',
 '.',
 'some',
 'tales',
 'told',
 'of',
 'how',
 'he',
 'had',
 'wings',
 'as',
 'dark',
 'as',
 'night',
 ',',
 'and',
 'of',
 'how',
 'he',
 'wore',
 'a',
 'crown',
 'of',
 'razor',
 'wire',
 'woven',
 'around',
 'four',
 'horns',
 '.',
 'dante',
 'had',
 'heard',
 'all',
 'the',
 'stories',
 'before',
 '.',
 'the',
 'only',
 'tale',
 'he',
 'cared',
 'for',
 'was',
 'the',
 'one',
 'that',
 'said',
 'the',
 'jailor',
 'could',
 'bleed',
 '.',
 'if',
 'the',
 'jailor',
 'could',
 'bleed',
 

In [19]:
df['Genres1']

0         Romance
1          Horror
2         Romance
3         Romance
4          Horror
          ...    
901     Self Help
902    Historical
903     Self Help
904     Self Help
905    Historical
Name: Genres1, Length: 906, dtype: string

In [20]:
le = LabelEncoder()
df['Genres1'] = le.fit_transform(df['Genres1'])
print(le.classes_)

['Classics' 'Fantasy' 'Historical' 'Horror' 'Mystery' 'Romance'
 'Science Fiction' 'Self Help' 'Thriller' 'Young Adult']


In [21]:
df['Genres1']

0      5
1      3
2      5
3      5
4      3
      ..
901    7
902    2
903    7
904    7
905    2
Name: Genres1, Length: 906, dtype: int32

In [22]:
x_train, x_test, y_train, y_test = train_test_split(df['desc_final'], df['Genres1'], test_size = 0.2, shuffle=True)

In [23]:
Tfidf_vect = TfidfVectorizer(max_features=2000)
Tfidf_vect.fit(df['desc_final'])
Train_X_Tfidf = Tfidf_vect.transform(x_train)
Test_X_Tfidf = Tfidf_vect.transform(x_test)

In [24]:
model = svm.SVC(probability=True)
model.fit(Train_X_Tfidf, y_train)
svm_pred = model.predict(Test_X_Tfidf)
print(classification_report(y_test, svm_pred))

              precision    recall  f1-score   support

           0       0.69      0.82      0.75        22
           1       0.78      0.52      0.62        27
           2       0.00      0.00      0.00         3
           3       0.77      0.42      0.54        24
           4       0.83      0.50      0.62        20
           5       0.41      0.65      0.50        20
           6       0.53      0.75      0.62        12
           7       0.93      0.93      0.93        14
           8       0.42      0.69      0.52        16
           9       0.54      0.54      0.54        24

    accuracy                           0.61       182
   macro avg       0.59      0.58      0.57       182
weighted avg       0.65      0.61      0.61       182



In [25]:
print(le.classes_)

['Classics' 'Fantasy' 'Historical' 'Horror' 'Mystery' 'Romance'
 'Science Fiction' 'Self Help' 'Thriller' 'Young Adult']
